In [1]:
import pandas as pd
import numpy as np

In [2]:
toUser_train_data = pd.read_csv('./data/toUser_train_data.csv',index_col=False)
toUser_train_data.head(5)

if '用户标识' in toUser_train_data.columns.tolist():
    toUser_train_data = toUser_train_data.drop('用户标识', axis=1)
    toUser_train_data = toUser_train_data.drop('标签月', axis=1)

In [3]:
def check_data(toUser_train_data):
    for column in toUser_train_data.columns:
        unique_values = toUser_train_data[column].unique()
        print(column, "的唯一值：", unique_values)
# check_data(toUser_train_data)

# 删除逆天的值

In [4]:
toUser_train_data['年龄'].replace(-1, np.nan, inplace=True)

toUser_train_data['性别'].replace(9, np.nan, inplace=True)
toUser_train_data['性别'].replace(-1, np.nan, inplace=True)

# 编码

In [5]:
def coding(df):
    # 定义需要编码的列
    columns_to_check = [
        '当前终端品牌',
        '上次终端品牌',
        '上上次终端品牌',
        '家庭交往圈终端品牌偏好',
        '用户历史终端品牌偏好',
        '集团交往圈终端品牌偏好',
        '新5G终端品牌'
    ]

    # 定义类别和对应整数的字典
    category_mapping = {
        '华为': 1,
        '苹果': 2,
        'vivo': 3,
        'OPPO': 4,
        '荣耀': 5,
        '小米': 6,
        '三星': 7,
        '魅族': 8,
        '金立': 9,
        '中兴': 10,
        '一加': 11,
        '中国移动': 12,
            '其他': 0
    }

    # 循环处理每一列并进行编码
    for column in columns_to_check:
        if column in df.columns:
            df[column] = df[column].replace(category_mapping)
    return df

In [6]:
toUser_train_data = coding(toUser_train_data)

In [7]:
print(toUser_train_data['新5G终端品牌'])

0         11
1          1
2          4
3          1
4          4
          ..
557653     1
557654     7
557655     1
557656     1
557657     1
Name: 新5G终端品牌, Length: 557658, dtype: int64


# 缺失值填充

In [8]:
def not_null(df):
    non_null_columns = df.columns[df.notnull().all()]
    non_null_columns_list = non_null_columns.tolist()
    
    return non_null_columns_list


In [9]:
not_null_A = not_null(toUser_train_data)

In [10]:
# 查看每列的缺失值个数，仅输出缺失值个数不为 0 的列
missing_values = toUser_train_data.isna().sum()
miss_ls = []

for column, value in missing_values.items():
    if value != 0:
        miss_ls.append(column)
#       print(f'{column}: {value}')

print(miss_ls)

['年龄', '性别', '上月主套餐费用', '上月主套内包含流量', '上月ARPU', '上上月ARPU', '上上上月ARPU', '历史终端使用平均时长', '近三年换终端次数', '当前终端品牌', '当前终端价格', '当前终端价格档次', '当前终端是否双卡槽', '当前终端使用时长', '上次终端品牌', '上次终端价格', '上次终端价格档次', '上次终端是否双卡槽', '上次终端使用时长', '上上次终端品牌', '上上次终端价格', '上上次终端价格档次', '上上次终端是否双卡槽', '上上次终端使用时长', '上月DOU', '上上月DOU', '上上上月DOU', '上上月流量饱和度', '上上上月流量饱和度', '上月限速次数', '上月2', '近三月限速次数', '近三月2', '近3月月均上网天数', '家庭交往圈终端品牌偏好']


# 定义填充的函数

In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


def fill(ls, df):
    for i in ls:
        print('开始填充:',i)
        
        if df[i].nunique() < 15:
            clf = DecisionTreeClassifier()
        else:
            clf = DecisionTreeRegressor()

        train = df.dropna(subset=[i])
        test = df[df[i].isnull()]

        X_train = train[not_null_A]
        y_train = train[i]
        X_test = test[not_null_A]

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        df.loc[test.index, i] = y_pred

    if df.isna().sum().sum() == 0:
        print('已填充完成所有缺失值')
    else:
        print('仍有缺失值未填充')

    return df


In [12]:
toUser_train_data = fill(miss_ls,toUser_train_data)

开始填充: 年龄
开始填充: 性别
开始填充: 上月主套餐费用
开始填充: 上月主套内包含流量
开始填充: 上月ARPU
开始填充: 上上月ARPU
开始填充: 上上上月ARPU
开始填充: 历史终端使用平均时长
开始填充: 近三年换终端次数
开始填充: 当前终端品牌
开始填充: 当前终端价格
开始填充: 当前终端价格档次
开始填充: 当前终端是否双卡槽
开始填充: 当前终端使用时长
开始填充: 上次终端品牌
开始填充: 上次终端价格
开始填充: 上次终端价格档次
开始填充: 上次终端是否双卡槽
开始填充: 上次终端使用时长
开始填充: 上上次终端品牌
开始填充: 上上次终端价格
开始填充: 上上次终端价格档次
开始填充: 上上次终端是否双卡槽
开始填充: 上上次终端使用时长
开始填充: 上月DOU
开始填充: 上上月DOU
开始填充: 上上上月DOU
开始填充: 上上月流量饱和度
开始填充: 上上上月流量饱和度
开始填充: 上月限速次数
开始填充: 上月2
开始填充: 近三月限速次数
开始填充: 近三月2
开始填充: 近3月月均上网天数
开始填充: 家庭交往圈终端品牌偏好
已填充完成所有缺失值


In [17]:
# toUser_train_data.to_csv('./filled_data_A.csv',index=False)

In [18]:
print()

# 训练模型(以lightGBM为例)

In [19]:
import pandas as pd
import numpy as np

encoded_data = pd.read_csv('./filled_data_A.csv',index_col=False)

print(encoded_data.shape)

(557658, 68)


In [20]:
# 早停版
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 假设你已经有了经过编码的特征和目标变量
X_encoded = encoded_data.drop('新5G终端品牌', axis=1).values
y = encoded_data['新5G终端品牌'].values.astype(int)  # 将目标变量转换为整型

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

print(X_train.shape)

(446126, 67)


In [21]:
# 创建 LightGBM 数据集
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 设置模型参数
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 13,  # 设置类别的数量
    'num_leaves': 100,
    'learning_rate': 0.2,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': 10,
    'verbose': -1
}

print(X_test.shape)

# 训练模型
num_iterations = 20

model = lgb.train(params, lgb_train, num_boost_round=num_iterations, valid_sets=[lgb_train, lgb_test],
                  early_stopping_rounds=3, verbose_eval=True)

# 获取最佳迭代次数
best_iteration = model.best_iteration

# 预测
y_pred = model.predict(X_test, num_iteration=best_iteration)

# 保存最佳模型
model.save_model("lightgbm_model.txt", num_iteration=best_iteration)

(111532, 67)


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's multi_logloss: 1.29484	valid_1's multi_logloss: 1.32061
Training until validation scores don't improve for 3 rounds
[2]	training's multi_logloss: 1.31452	valid_1's multi_logloss: 1.35816
[3]	training's multi_logloss: 1.23306	valid_1's multi_logloss: 1.28276
[4]	training's multi_logloss: 1.20564	valid_1's multi_logloss: 1.26145
[5]	training's multi_logloss: 1.17855	valid_1's multi_logloss: 1.23406
[6]	training's multi_logloss: 1.16382	valid_1's multi_logloss: 1.22084
[7]	training's multi_logloss: 1.1367	valid_1's multi_logloss: 1.20875
[8]	training's multi_logloss: 1.15619	valid_1's multi_logloss: 1.23288
[9]	training's multi_logloss: 1.11608	valid_1's multi_logloss: 1.20576
[10]	training's multi_logloss: 1.11908	valid_1's multi_logloss: 1.21649
[11]	training's multi_logloss: 1.11528	valid_1's multi_logloss: 1.21771
[12]	training's multi_logloss: 1.12749	valid_1's multi_logloss: 1.2408
Early stopping, best iteration is:
[9]	training's multi_logloss: 1.11608	valid_1's mult

In [22]:
print("训练完成!")

训练完成!


## 评估模型

In [23]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# 加载已保存的模型
model = lgb.Booster(model_file='lightgbm_model.txt')

# 使用测试集进行预测
y_pred = model.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred_labels)
print('Accuracy:', accuracy)

# 生成分类报告
report = classification_report(y_test, y_pred_labels)
print('Classification Report:\n', report)


Accuracy: 0.6117616468816125
Classification Report:
               precision    recall  f1-score   support

           0       0.03      0.05      0.04       257
           1       0.64      0.87      0.74     59349
           3       0.52      0.38      0.44     15633
           4       0.53      0.31      0.39     12253
           5       0.56      0.20      0.30     15395
           6       0.67      0.43      0.52      6366
           7       0.51      0.46      0.48       901
           8       0.36      0.45      0.40       341
          10       0.07      0.21      0.10        63
          11       0.41      0.35      0.37       958
          12       0.00      0.00      0.00        16

    accuracy                           0.61    111532
   macro avg       0.39      0.34      0.34    111532
weighted avg       0.60      0.61      0.58    111532



# 预测b数据

In [24]:
import pandas as pd
import numpy as np

In [25]:
test_B = pd.read_csv("./data/test_B.csv",index_col=False)
ID = test_B['用户标识'].tolist()

if '标签月' in test_B.columns.tolist() or '用户标识' in test_B.columns.tolist():
    test_B = test_B.drop('标签月', axis=1)
    test_B = test_B.drop('用户标识', axis=1)

# 编码

In [26]:
test_B = coding(test_B)
test_B.head(5)

,年龄,用户状态,性别,网龄,客户类型,城县农标志,是否4G客户,用户星级,同身份证下的号码数,是否全球通客户,...,上月视频类流量,上月短视频类流量,上月游戏类流量占比,上月音乐类流量占比,上月视频类流量占比,上月短视频类流量占比,是否订购5G套餐,用户常驻地是否在5G基站3公里范围,是否资费敏感客户,新5G终端档次
0,26,1,0,3,1,0,1,0,1,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,3
1,43,1,0,92,1,2,1,3,3,1,...,5.1188,6.4713,0.0000,0.0000,0.0004,0.0005,1,1,0,2
2,31,1,1,114,1,2,1,2,1,1,...,243.0376,1.7531,0.0001,0.0004,0.0307,0.0002,0,1,0,4
3,47,1,1,208,1,2,0,1,3,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,1,0,5
4,54,1,0,137,1,2,1,5,1,1,...,4612.1906,39.9408,0.0034,0.0000,0.3723,0.0032,1,0,0,2


# 缺失值填空

In [27]:
not_null_B = not_null(test_B)

In [28]:
# 查看每列的缺失值个数，仅输出缺失值个数不为 0 的列
missing_values = test_B.isna().sum()
miss_ls = []

for column, value in missing_values.items():
    if value != 0:
        miss_ls.append(column)
#         print(f'{column}: {value}')

print(miss_ls)

['上月主套餐费用', '上月主套内包含流量', '上月ARPU', '上上月ARPU', '上上上月ARPU', '历史终端使用平均时长', '近三年换终端次数', '当前终端品牌', '当前终端价格', '当前终端价格档次', '当前终端是否双卡槽', '当前终端使用时长', '上次终端品牌', '上次终端价格', '上次终端价格档次', '上次终端是否双卡槽', '上次终端使用时长', '上上次终端品牌', '上上次终端价格', '上上次终端价格档次', '上上次终端是否双卡槽', '上上次终端使用时长', '上月DOU', '上上月DOU', '上上上月DOU', '上上月流量饱和度', '上上上月流量饱和度', '上月限速次数', '上月2', '近三月限速次数', '近三月2', '近3月月均上网天数', '家庭交往圈终端品牌偏好']


In [29]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


def fill_B(ls, df):
    for i in ls:
        print('开始填充:',i)
        if df[i].nunique() < 15:
            clf = DecisionTreeClassifier()
        else:
            clf = DecisionTreeRegressor()

        train = df.dropna(subset=[i])
        test = df[df[i].isnull()]

        X_train = train[not_null_B]
        y_train = train[i]
        X_test = test[not_null_B]

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        df.loc[test.index, i] = y_pred

    if df.isna().sum().sum() == 0:
        print('已填充完成所有缺失值')
    else:
        print('仍有缺失值未填充')

    return df


In [30]:
test_B = fill_B(miss_ls,test_B)

开始填充: 上月主套餐费用
开始填充: 上月主套内包含流量
开始填充: 上月ARPU
开始填充: 上上月ARPU
开始填充: 上上上月ARPU
开始填充: 历史终端使用平均时长
开始填充: 近三年换终端次数
开始填充: 当前终端品牌
开始填充: 当前终端价格
开始填充: 当前终端价格档次
开始填充: 当前终端是否双卡槽
开始填充: 当前终端使用时长
开始填充: 上次终端品牌
开始填充: 上次终端价格
开始填充: 上次终端价格档次
开始填充: 上次终端是否双卡槽
开始填充: 上次终端使用时长
开始填充: 上上次终端品牌
开始填充: 上上次终端价格
开始填充: 上上次终端价格档次
开始填充: 上上次终端是否双卡槽
开始填充: 上上次终端使用时长
开始填充: 上月DOU
开始填充: 上上月DOU
开始填充: 上上上月DOU
开始填充: 上上月流量饱和度
开始填充: 上上上月流量饱和度
开始填充: 上月限速次数
开始填充: 上月2
开始填充: 近三月限速次数
开始填充: 近三月2
开始填充: 近3月月均上网天数
开始填充: 家庭交往圈终端品牌偏好
已填充完成所有缺失值


In [32]:
# test_B.to_csv('./filled_data_B.csv',index=False)
# if encoded_data.isna().sum().sum() == 0:
#     print('已填充完成所有缺失值')
# else:
#     print('仍有缺失值未填充')

# 预测

In [33]:
test_B = pd.read_csv('./filled_data_B.csv',index_col=False)
print(test_B.shape)

(30981, 67)


In [34]:
import lightgbm as lgb
import pandas as pd

# 加载保存的模型
model = lgb.Booster(model_file='lightgbm_model.txt')

X_new_encoded = test_B.values

# 进行预测
y_new_pred = model.predict(X_new_encoded)

predicted_classes = y_new_pred.argmax(axis=1)

predicted_classes

array([3, 3, 1, ..., 1, 1, 5])

In [35]:
result = pd.DataFrame({'用户标识': ID, '新5G终端品牌': predicted_classes})
result.head(5)

,用户标识,新5G终端品牌
0,4029657c43fb8daa04b5bd5c55f149bf,3
1,644b29afecb6f48a9d630c3195dedff8,3
2,862a21b8f03df807b8e5e0a4f34ae86d,1
3,f662c936f4d9af1bf7c5ff3faaf42a38,1
4,b2dd5190896b01ced813918773c18b30,1


In [36]:
column_name = '新5G终端品牌'
value_counts = result[column_name].value_counts()
print(value_counts)

1     22116
3      2889
4      1772
5      1516
6      1464
7       598
11      281
8       157
0       136
10       45
12        7
Name: 新5G终端品牌, dtype: int64


# 反向编码

In [37]:
def reverse_encoding(df):
    # 定义需要反向编码的列
    columns_to_reverse = [
        '新5G终端品牌'
    ]

    # 定义整数和对应类别的字典
    reversed_mapping = {
        1: '华为',
        2: '苹果',
        3: 'vivo',
        4: 'OPPO',
        5: '荣耀',
        6: '小米',
        7: '三星',
        8: '魅族',
        9: '金立',
        10: '中兴',
        11: '一加',
        12: '中国移动',
        0: '其他'
    }

    # 循环处理每一列并进行反向编码
    for column in columns_to_reverse:
        if column in df.columns:
            df[column] = df[column].replace(reversed_mapping)
    return df
result = reverse_encoding(result)


In [38]:
result.head(5)

,用户标识,新5G终端品牌
0,4029657c43fb8daa04b5bd5c55f149bf,vivo
1,644b29afecb6f48a9d630c3195dedff8,vivo
2,862a21b8f03df807b8e5e0a4f34ae86d,华为
3,f662c936f4d9af1bf7c5ff3faaf42a38,华为
4,b2dd5190896b01ced813918773c18b30,华为


# 保存

In [39]:
result.to_csv('./Submit.csv',index=False)

In [42]:
!castlecli --third honghu --token 752a086cdf4e3be334473f63c7318c8b --source Submit.csv


〔 版本状态 〕 v0.14 
〔 校验权限 〕 有效 
〔 竞赛信息 〕 第三届中国移动“梧桐杯”数据应用赛道 
〔 用户信息 〕 18939006986 
〔 提交备注 〕  
〔 文件分段 〕 [1]段 
〔 上传进度 〕 [##################################################]100%        1/1
〔 资源标识 〕 0000018AB1B1915E804F2BC8EBD8A841 
〔 提交状态 〕 失败(初赛评审材料提交时间还没开始,暂不可提交)  

